In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from torchmetrics import StructuralSimilarityIndexMeasure
from statistics import median, mean
from matplotlib import pyplot as plt
import numpy as np
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from torchsummary import summary
import json
from tqdm import tqdm
from glob import glob
import sys
path = "/study/mrphys/skunkworks/kk/mriUnet"
sys.path.insert(0,path)
import unet
from torchvision import transforms
from torch.utils.data import Dataset
import h5py
from sklearn.model_selection import KFold as kf
import os

allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))

In [2]:
def slice2d(array, discardZero=False):
    '''
    slice a 4d array of shape (c-channel, n, n, n) where n in the cube length
    into 3d arrays slices of shape (c, n, n) per each 2d plane
    '''
    result = []
    c, w, h, d = array.shape
    assert (w==h)and(h==d)and(d==w), f"Array must be cubic, got: {w}x{h}x{d}"
    for i in range(w):
        result.append(array[:,i,:,:])
        result.append(array[:,:,i,:])
        result.append(array[:,:,:,i])
    return np.array(result)

In [4]:
def getComplexSlices(path, return_scale=False):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            imagestackReal.append(np.array(image['real']))
            imagestackImag.append(np.array(image['imag']))
            if i==0:
                normScale = np.abs(np.array(image['real']+image['real']*1j)).max()
        imagestackReal = np.array(imagestackReal)/normScale
        imagestackImag = np.array(imagestackImag)/normScale
        imagesliceReal = slice2d(imagestackReal)
        imagesliceImag = slice2d(imagestackImag)
        
    if return_scale:
        return imagesliceReal+imagesliceImag*1j, normScale
    else:
        return imagesliceReal+imagesliceImag*1j

In [5]:
class mriSliceDataset(Dataset):
    def __init__(self, sample):
        self.originalPathList = []
        self.accelPathList = []
        self.originalFileList = []
        self.accelFileList = []

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
        folderName  = allImages[sample]
        self.originalPathList.append(folderName + 'processed_data/C.h5')
        self.accelPathList.append(folderName +'processed_data/acc_2min/C.h5')
        
        for originalPath, accelPath in zip(self.originalPathList, self.accelPathList):
            self.originalFileList+= list(getComplexSlices(originalPath))
            self.accelFileList+= list(getComplexSlices(accelPath))
            print('Image ' + originalPath + ' loaded')

    def __getitem__(self, index):
        return self.accelFileList[index], self.originalFileList[index]

    def __len__(self):
        return len(self.accelFileList)

In [6]:
l

  0%|                                                                                            | 0/65 [00:00<?, ?it/s]

Image /study/mrphys/skunkworks/training_data//mover01/M001/processed_data/C.h5 loaded


  2%|█▎                                                                                  | 1/65 [00:25<27:07, 25.43s/it]

Image /study/mrphys/skunkworks/training_data//mover01/M002/processed_data/C.h5 loaded


  3%|██▌                                                                                 | 2/65 [00:51<26:52, 25.60s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████▍  | 63/65 [29:22<00:59, 29.52s/it]

Image /study/mrphys/skunkworks/training_data//mover01/M069/processed_data/C.h5 loaded


 98%|█████████████████████████████████████████████████████████████████████████████████▋ | 64/65 [29:51<00:29, 29.26s/it]

Image /study/mrphys/skunkworks/training_data//mover01/M070/processed_data/C.h5 loaded


100%|███████████████████████████████████████████████████████████████████████████████████| 65/65 [30:20<00:00, 28.01s/it]


In [ ]:
class noisyDataset(Dataset):
    def __init__(self, sample):
        self.accelPathList = []
        self.accelFileList = []

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
        folderName  = allImages[sample]
        self.accelPathList.append(folderName +'processed_data/acc_2min/C.h5')
        
        for accelPath in self.accelPathList:
            slices, scale = getComplexSlices(accelPath, return_scale=True)
            self.accelFileList+= list(slices)
            self.scale = scale
            print('Image ' + accelPath + ' loaded')

    def __getitem__(self, index):
        return self.accelFileList[index]

    def __len__(self):
        return len(self.accelFileList)
    
def predict(model, dataset, device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    model.eval()
    model.to(device)
    X = []
    Y = []
    Z = []
    for i, noisy in tqdm(enumerate(dataset)):
        noisy = torch.tensor(noisy).to(device).unsqueeze(0)
        with torch.no_grad():
            p = model(noisy).cpu().numpy() * dataset.scale
            if i%3==0:
                X.append(p)
            elif i%3==1:
                Y.append(p)
            else:
                Z.append(p)
                
    return np.vstack(X).transpose(1,0,2,3), np.vstack(Y).transpose(1,2,0,3), np.vstack(Z).transpose(1,2,3,0)

def get_prediction(idx, fold):
    
    model = unet.UNet(6,
    6,
    f_maps=32,
    layer_order=['separable convolution', 'relu'],
    depth=3,
    layer_growth=2.0,
    residual=True,
    complex_input=True,
    complex_kernel=True,
    ndims=2,
    padding=1)

    name = f'slice_kfold_{fold}'

    model.load_state_dict(torch.load(f'/study/mrphys/skunkworks/kk/outputs/{name}/weights/{name}_LATEST.pth'))

    dataset = noisyDataset(idx)
    X, Y, Z = predict(model, dataset)
    
    return (X+Y+Z)/3

In [ ]:
pred = get_prediction(4, 1)

In [ ]:
plt.gray()
n = 130
plt.title("Top-down")
plt.imshow(pred[0][n,:,:].imag)
plt.show()
plt.title("Left-right")
plt.imshow(pred[0][:,n,:].imag)
plt.show()
plt.title("Front-back")
plt.imshow(pred[0][:,:,n].imag)
plt.show()

In [ ]:
kfsplitter = kf(n_splits=5, shuffle=True, random_state=69420)
for i, (train_index, test_index) in enumerate(kfsplitter.split(np.arange(65))):
    for idx in test_index:
        print(f'Fold = {i+1}')
        pred = get_prediction(idx, i+1)
        np.save(f'pred/denoised_{idx}.npy', pred)

In [ ]:
path = '/study/mrphys/skunkworks/training_data//mover01/M001/processed_data/acc_2min/C.h5'

with h5py.File(path,'r') as hf:
    prefix = 'C_000_0'
    imagestackReal = []
    imagestackImag = []
    for i in range(6):
        n = prefix + str(i).zfill(2)
        image = hf['Images'][n]
        imagestackReal.append(np.array(image['real']))
        imagestackImag.append(np.array(image['imag']))
        if i==0:
            normScale = np.max([np.abs(np.array(image['real'])).max(), np.abs(np.array(image['real'])).max()])
    imagestackReal = np.array(imagestackReal)/normScale
    imagestackImag = np.array(imagestackImag)/normScale
    x = imagestackReal+imagestackImag*1j
    
plt.imshow(x[0,130].real)

# Save to H5

In [ ]:
import h5py
import numpy as np

imgIndex = 0
name = allImages[imgIndex].split('/')[-2]
with h5py.File(f'/scratch/mrphys/denoised/comparison_{name}.h5','w') as f:
    grp = f.create_group('Original')
    with h5py.File(allImages[imgIndex]+'processed_data/C.h5','r') as hfOriginal:
        for n in range(6):
            n = 'C_000_0'+ str(n).zfill(2)
            grp.create_dataset(n, data=np.array(hfOriginal['Images'][n]))
        
    grp = f.create_group('Noisy')
    with h5py.File(allImages[imgIndex]+'processed_data/acc_2min/C.h5','r') as hfNoisy:
        for n in range(6):
            n = 'C_000_0'+ str(n).zfill(2)
            grp.create_dataset(n, data=np.array(hfNoisy['Images'][n]))

    pred = np.load(f'pred/denoised_{0}.npy')
    temp = pred.astype(np.dtype([('real','f'),('imag','f')]))
    temp['imag'] = pred.imag
    pred = temp
    grp = f.create_group('Denoised')
    for n in range(6):
        grp.create_dataset('C_000_0'+ str(n).zfill(2), data=pred[n])

In [ ]:
with h5py.File(f'/scratch/mrphys/denoised/comparison_{name}.h5','r') as f:
    n = 0
    n = 'C_000_0'+ str(n).zfill(2)
    plt.imshow(f['Denoised'][n]['real'][130])
    plt.show()
    plt.imshow(f['Original'][n]['real'][130])
    plt.show()
    plt.imshow(f['Noisy'][n]['real'][130])
    plt.show()

# Save using the same format

In [7]:
import h5py
import numpy as np
import os
from glob import glob
from tqdm import tqdm

In [8]:
allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
try:
    os.mkdir('/scratch/mrphys/denoised')
except:
    pass

for i, imgIndex in tqdm(enumerate(range(len(allImages)))):
    name = allImages[imgIndex].split('/')[-2]
    with h5py.File(f'/scratch/mrphys/denoised/denoised_{name}.h5','w') as f:
        pred = np.load(f'pred/denoised_{i}.npy')
        temp = pred.astype(np.dtype([('real','f'),('imag','f')]))
        temp['imag'] = pred.imag
        pred = temp
        grp = f.create_group('Images')
        for n in range(6):
            grp.create_dataset('C_000_0'+ str(n).zfill(2), data=pred[n])

0it [00:00, ?it/s]/tmp/ipykernel_1749358/3271076045.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  temp = pred.astype(np.dtype([('real','f'),('imag','f')]))
65it [02:22,  2.19s/it]


In [ ]:
with h5py.File(path,'r') as f:
    print(f.keys())
    print(f['Images']['C_000_001'])

In [ ]:
with h5py.File(f'/scratch/mrphys/skunkworks/denoised/denoised_{name}.h5','r') as f:
    print(f.keys())
    print(f['Images']['C_000_001'])

In [ ]:
import torch.distributed as dist
import torch
import os

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '6969'
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()
    
n_gpus = torch.cuda.device_count()
rank = 4
print(rank)
setup(rank, n_gpus)
rank = dist.get_rank()
print(rank)

In [5]:
import pickle
from torch.utils.data import Dataset
class mriSliceDataset(Dataset):
    def __init__(self, sample):
        self.originalPathList = []
        self.accelPathList = []
        self.originalFileList = []
        self.accelFileList = []

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
        folderName  = allImages[sample]
        self.originalPathList.append(folderName + 'processed_data/C.h5')
        self.accelPathList.append(folderName +'processed_data/acc_2min/C.h5')
        
        for originalPath, accelPath in zip(self.originalPathList, self.accelPathList):
            self.originalFileList+= list(getComplexSlices(originalPath))
            self.accelFileList+= list(getComplexSlices(accelPath))
            print('Image ' + originalPath + ' loaded')

    def __getitem__(self, index):
        return self.accelFileList[index], self.originalFileList[index]

    def __len__(self):
        return len(self.accelFileList)
with open(f'/scratch/mrphys/pickled/dataset_{0}.pickle', 'rb') as f:
    data = pickle.load(f)

In [6]:
for X, Y in data:
    print(X.shape)

(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 256)
(16, 256, 

In [3]:
import sys
sys.path.insert(0,"/study3/mrphys/skunkworks/kk/mriUnet")
import unet
import torch
from torch import nn
model = unet.PatchGAN(
    14,
    f_maps=32,
    layer_order=['separable convolution', 'relu', 'batch norm'],
    depth=4,
    layer_growth=2.0,
    residual=True,
    complex_input=False,
    complex_kernel=False,
    ndims=2,
    padding=1
)

In [4]:
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inp = torch.randn(1,7,256,256)
inp = inp.to(device)
pred = model(inp, inp)

In [4]:
pred

tensor([[[[-2.7383e-01, -3.1528e-01, -1.0539e+00,  ..., -2.9841e-01,
           -4.1028e-02, -8.5437e-02],
          [ 1.3807e-01,  3.1959e-01, -1.9353e-02,  ...,  6.0839e-02,
           -5.8703e-01,  1.2111e-04],
          [-8.2378e-01,  5.7863e-02,  1.4145e+00,  ..., -2.4023e-02,
           -1.4754e-01, -6.0467e-01],
          ...,
          [-1.1502e-01, -2.8118e-01,  6.2925e-02,  ...,  2.1677e-01,
            6.0013e-01, -7.3894e-01],
          [ 2.2050e-02,  7.1407e-01,  5.9331e-02,  ...,  1.6346e-01,
           -6.3900e-01, -1.0993e+00],
          [-2.7264e-01,  2.7381e-02, -8.0088e-01,  ...,  3.9469e-01,
           -4.2766e-01, -4.0391e-01]]]], grad_fn=<ConvolutionBackward0>)

In [6]:
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
y = inp
ssim_loss = (1-ms_ssim(pred[:,:,:,:,0], y.real, data_range=1, size_average=False)).mean() + (1-ms_ssim(pred[:,:,:,:,1], y.imag, data_range=1, size_average=False)).mean()

In [7]:
!nvidia-smi

Tue Mar 21 13:12:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:01:00.0 Off |                  Off |
| 43%   69C    P2              256W / 260W|  42793MiB / 49152MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--